# Training Dataset Normalization Parameters

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import required libraries

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
from datetime import datetime
from datetime import date
import os
import torchvision.models as tmodels

### Compute Dataset Normalization Parameters

In [ ]:
# Prepare dataframe to store parameters
norm_param_column_list = ['Dataset','meanR','meanG','meanB', 'stdR', 'stdG', 'stdB']
norm_param_df = pd.DataFrame(columns=norm_param_column_list)

In [ ]:
# Pre-load dataset without normalisation (normalisation parameters to be computed below)
for trg_dataset_ref in range (3):
    dataset_prenorm = datasets.ImageFolder(
        '/content/drive/MyDrive/KASHIKO/DATASET/TRG_' + str(trg_dataset_ref + 1) + '_FINAL/',
        transforms.Compose([
            transforms.ToTensor(),
        ])
    )
    # Compute normalisation parameters (i.e. mean and standard error for each channel)
    loader = torch.utils.data.DataLoader(dataset_prenorm,
                         batch_size=len(dataset_prenorm),
                         shuffle=False,
                         num_workers=4)
    mean = 0.
    std = 0.
    for images, _ in loader:
        batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

    mean /= len(loader.dataset)
    std /= len(loader.dataset)
    
    temp_df = pd.DataFrame([[trg_dataset_ref+1,
                             mean[0].item(),
                             mean[1].item(),
                             mean[2].item(),
                             std[0].item(),
                             std[1].item(),
                             std[2].item()]],
                             columns=norm_param_column_list)
    norm_param_df = norm_param_df.append(temp_df)

temp_df = pd.DataFrame([["AVG",
                          norm_param_df['meanR'].mean(),
                          norm_param_df['meanG'].mean(),
                          norm_param_df['meanB'].mean(),
                          norm_param_df['stdR'].mean(),
                          norm_param_df['stdG'].mean(),
                          norm_param_df['stdB'].mean()]],
                         columns=norm_param_column_list)
norm_param_df.to_csv('/content/drive/MyDrive/KASHIKO/DATASET/TRG_DATASET_NORM_PARAM.csv')